In [42]:
from PIL import Image
import numpy as np
image_path='./图片.png'
image=Image.open(image_path)
print(f"image_size：{image.size}")

image_size：(384, 384)


In [43]:
cols,rows=image.size
image=image.convert('L')
image_array=np.array(image)
print(image_array)

[[159 145 158 ... 149 155 149]
 [157 161 154 ... 152 150 154]
 [156 148 162 ... 147 154 150]
 ...
 [162 159 165 ... 149 156 145]
 [155 168 151 ... 158 145 149]
 [156 162 151 ... 158 152 167]]


In [44]:
gray_levels:np.array=np.zeros(256,dtype=int)
for row_array in image_array:
    for col_gray_level in row_array:
        gray_levels[col_gray_level]=gray_levels[col_gray_level]+1
print(gray_levels)

[ 696    3    3    9   23   39   70  120  201  324  453  659  989 1257
 1566 1952 2299 2594 2869 3024 2910 2879 2634 2395 2047 1566 1281  977
  717  497  362  217  127   77   49   25   13    5    1    2    1    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    5    6   18   38   90
  122  228  407  624  991 1438 2002 2813 3791 4777 5793 6791 7796 8134
 8735 8623 8219 7650 6729 5707 4711 3672 2810 2072 1440 1004  625  384
  235  118  101   45   22   11    5    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [45]:
gray_levels[0]=0
gray_levels[255]=0

可以将前半段视为前景，将后半段视为背景？
前景，数量最多的像素位置在19（3024）；背景，数量最多的像素位置在154（8735）

In [46]:
back_max_position=np.argmax(gray_levels)
tmp_divide=back_max_position//2
front_max_position=np.argmax(gray_levels[0:tmp_divide])
print(f"back_max_position：{back_max_position}")
print(f"front_max_position：{front_max_position}")
print(tmp_divide)

back_max_position：154
front_max_position：19
77


对背景进行估计（最大似然）

In [47]:
def calculate(level_arr:np.ndarray,gray_move=0)->list:
    num=0
    gray_sum=0.0
    square_sum=0.0
    for gray_level,level_num in enumerate(level_arr):
        gray_level=gray_level+gray_move
        num=num+level_num
        gray_sum=gray_sum+level_num*gray_level
    u=gray_sum/num
    for gray_level,level_num in enumerate(level_arr):
        gray_level=gray_level+gray_move
        tmp_diff=gray_level-u
        square_sum=square_sum+level_num*tmp_diff*tmp_diff
    var=square_sum/num
    return [u,var]

In [48]:
[front_u,front_var]=calculate(gray_levels[tmp_divide:256],gray_move=tmp_divide)
[back_u,back_var]=calculate(gray_levels[0:tmp_divide])
print(f"前景像素灰度平均值为：{front_u}")
print(f"前景像素灰度方差为：{front_var}")
print(f"后景像素灰度平均值为：{back_u}")
print(f"后景像素灰度方差为：{back_var}")

前景像素灰度平均值为：154.48991561103858
前景像素灰度方差为：25.21353199449642
后景像素灰度平均值为：19.576887957890214
后景像素灰度方差为：24.717662202614672


舍友提出了使用最小二乘法拟合的方法